## Séparation du jeu

L'objectif de ce notebook est de séparer le jeu de données en un jeu d'entraînement (90% des données) et un jeu de validation (10% des données).
Le code de ce notebook sera extrait et exécuté sur la machine Tesla afin de profiter du CPU et de la mémoire disponible pour créer deux nouveaux fichiers h5 tout en gardant le dataset original en mémoire et intact.
La séparation du jeu en fichiers distincts permettra de s'assurer facilement par la suite que l'on s'entraîne uniquement sur le jeu d'entraînement et qu'on ne voit jamais les données de validation avant la phase de test

Une fois le jeu séparé en un jeu d'entraînement et un jeu de validation, on créé un sous ensemble du jeu d'entraînement sur lequel on va pouvoir par la suite faire les tests d'entraînement de modèles avant de lancer l'entraînement des modèles sur l'ensemble d'un jeu d'entraînement. On note que de cette façon on n'aura jamais vu les données du jeu de test.

### Définition des constantes

In [4]:
original_dataset_filename = "../data/riken_v2.h5"
train_dataset_filename = "../data/train_set_riken_v2.h5"
test_dataset_filename = "../data/test_set_riken_v2.h5"
minimal_dataset_filename = "../data/minimal_set_riken_v2.h5"
test_size = 0.1
minimal_dataset_size = 100000

### Séparation jeu d'entraînement/jeu de test

#### Définition de la fonction de séparation du jeu original en un jeu d'entraînement et un jeu de test

In [1]:
from sklearn.model_selection import train_test_split
import time
import h5py
import numpy as np

def split_jeux(original_dataset_filename, train_dataset_filename, test_dataset_filename, test_size=0.1):
    """ Fonction prenant un dataset h5 en entrée et le séparant en un jeu d'entraînement et un jeu de test,
    qu'elle enregistre"""

    start_time = time.time()
    
    # On charge le jeu de données original (en lecture seule)
    original_dataset = h5py.File(original_dataset_filename, 'r')  

    # On créé un tableau numpy contenant toutes les données
    print("Loading the data...")
    data = np.array([np.int32(original_dataset["pubchem_id"]), 
              np.asarray(original_dataset["anums"]),
              np.asarray(original_dataset["amasses"]),
              np.asarray(original_dataset["riken_coords"])]).T
    
    # On fait appel à scikit learn pour séparer les données de façon aléatoire
    print("Separation of the data...")
    train_set, test_set = train_test_split(data, test_size=test_size, random_state = 12)
    
    
    # On créé deux fichiers h5
    print("Creating h5 files...")
    train_set_h5 = h5py.File(train_dataset_filename, 'w')
    test_set_h5 = h5py.File(test_dataset_filename, 'w')

    try:
        # Définition du type pour les tableaux de floats
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))

        # Calcul de la taille des deux datasets de sortie
        train_size = len(train_set)
        test_size = len(test_set)
        
        col_names = ["pubchem_id", "anums", "amasses", "riken_coords"]
        datatypes = [np.int32, varlen_floatarray, varlen_floatarray, varlen_floatarray]

        train_datasets = []
        test_datasets = []
        
        # On créé les quatre datasets (un par colonne) par fichier de sortie
        print("Creating datasets...")
        for i in range(4):
            
            train_datasets.append(train_set_h5.create_dataset(col_names[i], shape=(train_size,),
                                       dtype=datatypes[i], compression="gzip", 
                                       chunks=True, maxshape=(None,)))

            test_datasets.append(test_set_h5.create_dataset(col_names[i], shape=(test_size,),
                                       dtype=datatypes[i], compression="gzip", 
                                       chunks=True, maxshape=(None,)))

            
            
        # On écrit le dataset d'entraînement en mémoire
        print("Writing train set in memory")
        for i in range(train_size):
            j=0
            for col_name in col_names:
                train_datasets[j][i] = train_set[i][j]
                j += 1
        
        # On écrit le dataset de test en mémoire
        print("Writing test set in memory")
        for i in range(test_size):
            j=0
            for col_name in col_names:
                test_datasets[j][i] = test_set[i][j]
                j += 1


        
        # On écrit les datasets sur le disque
        print("Writing train set to disk...")
        train_set_h5.flush()

        print("Writing test set to disk...")
        test_set_h5.flush()
        
        print("Succesful creation of a train set and a test set")
        print("--- %s seconds ---" % (time.time() - start_time))


        
    finally:
        # Closing the files
        original_dataset.close()
        train_set_h5.close()
        test_set_h5.close()


#### Appel à la fonction de séparation du jeu original en un jeu d'entraînement et un jeu de test

In [3]:
# Appel à la fonction de séparation du jeu

print("SÉPARATION DU JEU ORIGINAL EN UN JEU D'ENTRAÎNEMENT ET UN JEU DE TEST")
split_jeux(original_dataset_filename, train_dataset_filename, test_dataset_filename, test_size=test_size)


SyntaxError: invalid syntax (<ipython-input-3-300fc495dc68>, line 2)

### Séparation jeu d'entraînement/jeu d'entraînement minimal

#### Définition de la fonction de séparation

In [ ]:
import time

def creer_h5_ss_ensemble(original_dataset_filename, dataset_dest_filename, taille):
    
    start_time = time.time()

    # Ouverture du dataset d'entrée en lecture seule
    original_dataset =  h5py.File(original_dataset_filename, 'r')
    
    # Création du dataset minimal de sortie
    dataset_reduit_h5 = h5py.File(dataset_dest_filename, 'w')
    
    try:
        # Définition du type spécial
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))

        print("Copie des données...")
        
        # Copie des premières valeurs de la colonne pubchem_id
        dataset_reduit_h5.create_dataset("pubchem_id", shape=(taille,),
                                       dtype=np.int32, compression="gzip", 
                                       chunks=True, maxshape=(None,), 
                                       data=original_dataset["pubchem_id"][:taille])

        # Copie des premières valeurs des autres colonnes
        cols = ["anums", "amasses", "riken_coords"]  
        for col_name in cols:
            dataset_reduit_h5.create_dataset(col_name, shape=(taille,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True, maxshape=(None,), 
                                           data=original_dataset[col_name][:taille])

        dataset_reduit_h5.flush()
        print("Jeu minimal enregistré avec succès")
        print("--- %s seconds ---" % (time.time() - start_time))
        
    finally:
        dataset_reduit_h5.close()


#### Appel de la fonction de création d'un jeu d'entraînement minimal

In [ ]:
print()
print("CRÉATION JEU D'ENTRAÎNEMENT MINIMAL")

creer_h5_ss_ensemble(train_dataset_filename, minimal_dataset_filename, minimal_dataset_size)
